In [1]:
from  data_preprocessing import return_dataloaders
from computer_vision.auteencoders.models.VariationalAutoEncoder import VariationalAutoEncoder
import torch
import torch.nn as nn
import tqdm.notebook as tq
import matplotlib.pyplot as plt


In [2]:
train_dataloader, val_dataloader, test_dataloader = return_dataloaders(128, 128,8)
model =VariationalAutoEncoder()
#device = ('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

In [3]:
def train(model, dataloader, criterion, opt, epoch, device=device):
  model.train()
  epoch_loss = 0.0
  for image, _ in tq.tqdm(dataloader):
    image = image.to(device).view(-1, 784)
    output, mu, sigma  = model(image)
    reconst_loss = criterion(output, image)
    kl_div = - torch.sum(1 + torch.log(sigma.pow(2)) - mu.pow(2) - sigma.pow(2))

    loss = reconst_loss + kl_div

    opt.zero_grad()
    loss.backward()
    opt.step()

    epoch_loss += loss.item() * image.size(0)
  epoch_loss = epoch_loss / len(dataloader.dataset)
  print(f'Training, Epoch : {epoch} train_loss: {epoch_loss}')
  return model

In [4]:
model.to(device)
criterion = nn.BCELoss(reduction="sum")
lr = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.4)

In [5]:
NUM_EPOCHS = 4
for epoch in range(NUM_EPOCHS):
  model = train(model, train_dataloader, criterion, optimizer, epoch)

  0%|          | 0/469 [00:00<?, ?it/s]

Training, Epoch : 0 train_loss: 22643.639715625


  0%|          | 0/469 [00:00<?, ?it/s]

Training, Epoch : 1 train_loss: 21661.81672291667


  0%|          | 0/469 [00:00<?, ?it/s]

Training, Epoch : 2 train_loss: 26319.951775


  0%|          | 0/469 [00:00<?, ?it/s]

Training, Epoch : 3 train_loss: 26335.751840625
